<a href="https://colab.research.google.com/github/KimBbaoro/KEB_toy/blob/master/LSTM_Cell_parctice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
torch.manual_seed(125)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(125)

In [2]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(1.0, ) ) #평균, 표편
])

In [3]:
from torchvision.datasets import MNIST

root = "/content/sample_data"
train_dataset = MNIST(root , transform = mnist_transform, train = True, download =True)
valid_dataset = MNIST(root, transform = mnist_transform, train = False, download =True)
test_dataset = MNIST(root, transform = mnist_transform, train = False, download =True)

100%|██████████| 9912422/9912422 [00:00<00:00, 171879296.89it/s]


Extracting /content/sample_data/MNIST/raw/train-images-idx3-ubyte.gz to /content/sample_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 111440380.70it/s]


Extracting /content/sample_data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/sample_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 146765659.28it/s]

Extracting /content/sample_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/sample_data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 6442519.03it/s]


Extracting /content/sample_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/sample_data/MNIST/raw



In [4]:
batch_size = 64
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size,
                          shuffle = True)
valid_loader = DataLoader(dataset = test_dataset, batch_size = batch_size,
                          shuffle = True)

In [5]:
batch_size = 100
n_iters = 6000

#몇번 반복하는

num_epochs = n_iters/(len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

우리는 셀에 집중할 것임.//오류 class

#수정된 cell

In [6]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        # input에서 h로 갈
        # hidden to hidden
        # why *4? -> 각 게이트에 줄거기 때문에
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            # 난수 생성
            w.data.uniform_(-std, std)

    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(1))

        gates = self.x2h(x) + self.h2h(hx)
        gates = gates.squeeze()
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)

        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))

        return (hy, cy)


In [16]:
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim , layer_dim, output_dim, bias = True):
    super(LSTMModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim
    self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    if torch.cuda.is_available():
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
    else:
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

    if torch.cuda.is_available():
      c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
    else:
      c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

    outs = []

    hn = h0[0,:,:]
    cn = c0[0,:,:]

    for seq in range(x.size(1)):
      hn, cn = self.lstm(x[:,seq,:], (hn,cn))
      outs.append(hn)

    out = outs[-1].squeeze()
    out = self.fc(out)
    return out

옵티마이저와 손실함수 지정

In [17]:
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim , output_dim)
if torch.cuda.is_available():
  model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [22]:
seq_dim = 28 
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):          
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim))
            labels = Variable(labels)
          
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        if torch.cuda.is_available():
            loss.cuda()

        loss.backward()
        optimizer.step()        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 500 == 0:         
            correct = 0
            total = 0
            for images, labels in valid_loader:
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1 , seq_dim, input_dim))
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                 
                total += labels.size(0)
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.3325059711933136. Accuracy: 93.80999755859375
Iteration: 1000. Loss: 0.1143890768289566. Accuracy: 95.38999938964844
Iteration: 1500. Loss: 0.09057844430208206. Accuracy: 96.54000091552734
Iteration: 2000. Loss: 0.18395444750785828. Accuracy: 96.05999755859375
Iteration: 2500. Loss: 0.06006951630115509. Accuracy: 97.12999725341797
Iteration: 3000. Loss: 0.04892972111701965. Accuracy: 97.33000183105469
Iteration: 3500. Loss: 0.10136602073907852. Accuracy: 97.4800033569336
Iteration: 4000. Loss: 0.05847252532839775. Accuracy: 97.5
Iteration: 4500. Loss: 0.17699843645095825. Accuracy: 97.44000244140625
Iteration: 5000. Loss: 0.1935262233018875. Accuracy: 96.58000183105469
Iteration: 5500. Loss: 0.07306650280952454. Accuracy: 97.55999755859375
Iteration: 6000. Loss: 0.03944644704461098. Accuracy: 97.93000030517578
Iteration: 6500. Loss: 0.06581221520900726. Accuracy: 97.52999877929688
Iteration: 7000. Loss: 0.01685594767332077. Accuracy: 97.72000122070312
Iteration:

In [31]:
def evaluate(model, val_iter):
  corrects, total, total_loss = 0,0,0
  model.eval()
  for images, labels in val_iter:
    if torch.cuda.is_available():
      images = Variable(images.view(-1, seq_dim, input_dim).cuda())
    else:
      images = Variable(images.view(-1, seq_dim, input_dim)).to(device)
    labels = labels.cuda()
    logit = model(images).to(device)
    loss = F.cross_entropy(logit, labels, reduction = "sum")
    _, predicted = torch.max(logit.data, 1)

    total += labels.size(0)
    total_loss += loss.item()
    corrects +=(predicted == labels).sum()

  avg_loss = total_loss/len(val_iter.dataset)
  avg_accuracy = corrects/total
  return avg_loss, avg_accuracy

In [ ]:
def evaluate(model, val_iter):    
    corrects, total, total_loss = 0, 0, 0
    model.eval()
    for images, labels in val_iter:
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else:
            images = Variable(images.view(-1 , seq_dim, input_dim)).to(device)
        labels = labels.cuda()        
        logit = model(images).cuda()
        loss = F.cross_entropy(logit, labels, reduction = "sum")
        _, predicted = torch.max(logit.data, 1)
        total += labels.size(0)
        total_loss += loss.item()
        corrects += (predicted == labels).sum()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [32]:
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
test_loss, test_acc = evaluate(model, test_loader)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))


Test Loss:  0.07 | Test Accuracy:  0.98
